## Import the necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import nltk
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords


from sklearn.metrics import f1_score

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## Load in your data from kaggle.  
By working in a kaggle kernel, you can access the data directly from the competition, as well as make your submission without downloading your output file

In [ ]:
train = pd.read_csv('../input/climate-change-edsa2020-21/train.csv')
test = pd.read_csv('../input/climate-change-edsa2020-21/test.csv')

In [ ]:
print(train.shape)

In [ ]:
train.tail()

In [ ]:
test.head()

In [ ]:
train.info()


In [ ]:
test.info()

In [ ]:
train.sentiment.value_counts()

In [ ]:
train[train.sentiment==1].head(10)

## Splitting out the X variable from the target

In [ ]:
y = train['sentiment']
X = train['message']

In [ ]:
print(y)


In [ ]:
print(X)

In [ ]:
from nltk.stem import PorterStemmer

# initiate Stemmer
porter_stemmer=PorterStemmer()

def preprocessor(text):
    
    text=text.lower()
    text=re.sub(re.sub(r'\^[a-zA-Z]s+', '', text))
    text=re.sub("\\s+(in|the|all|for|and|on)\\s+"," _connector_ ",text)
    
    # stem words
    words=re.split("\\s+", text)
    stemmed_words=[porter_stemmer.stem(word=word) for word in words]
    return ''.join(stemmed_words)

In [ ]:
def text_tokenizer(text):
    # provide space between special characters
    text=re.sub("(\\W)"," \\1", text)
    
    # split on whitespace
    return re.split("\\s+",text)

## Turning text into something your model can read

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), tokenizer=text_tokenizer, min_df=2, stop_words=["english"],max_df=0.75)
X_vectorized = vectorizer.fit_transform(X)

#stop_words="english",max_df=0.85, preprocessor=my_cool_preprocessor,"all","in","the","is","and"

In [ ]:
#vectorizer.stop_words_

In [ ]:
print(X_vectorized)

## Splitting the training data into a training and validation set

In [ ]:
X_train,X_val,y_train,y_val = train_test_split(X_vectorized,y,test_size=0.2,shuffle=True, stratify=y, random_state=25)

## Training the model and evaluating using the validation set 

In [ ]:
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc_pred = rfc.predict(X_val)


In [ ]:
print(rfc_pred)

In [ ]:
svm_lsvc =LinearSVC()
svm_lsvc.fit(X_train, y_train)
svm_lsvc_pred = svm_lsvc.predict(X_val)

In [ ]:
print(svm_lsvc_pred)

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_val)

## Checking the performance of our model on the validation set

In [ ]:
f1_score(y_val, svm_lsvc_pred, average="macro")

In [ ]:
f1_score(y_val, rfc_pred, average="macro")

In [ ]:
f1_score(y_val, lr_pred, average="macro")

In [ ]:
from sklearn import metrics

print(metrics.classification_report(y_val, svm_lsvc_pred))

In [ ]:
print(metrics.classification_report(y_val, rfc_pred))

In [ ]:
print(metrics.classification_report(y_val, lr_pred))

## Getting our test set ready 

In [ ]:
testx = test['message']
test_vect = vectorizer.transform(testx)

## Making predictions on the test set and adding a sentiment column to our original test df

In [ ]:
y_pred = svm_lsvc.predict(test_vect)

In [ ]:
print(y_pred)

In [ ]:
test['sentiment'] = y_pred

In [ ]:
test.head()

## Creating an output csv for submission

In [ ]:
test[['tweetid','sentiment']].to_csv('testsubmission.csv', index=False)